# Retrieve NFTs Assets by Address using CardanoScan API
---
This notebook details the process of retrieving the NFTs associated with a specific Cardano address using the [CardanoScan API](https://docs.cardanoscan.io/operation/operation-get-asset-list-byaddress). The process includes securely accessing your API key, querying the API, and saving the results in a formatted JSON file.
---
### 1.1 **1.4 Necessary Package Installations and Open Statements**

In [ ]:
#r "nuget: FSharp.Data"

In [2]:
open System
open System.IO
open System.Text.Json
open System.Net.Http
open System.Threading.Tasks
open FSharp.Data

## 2. Workflow

*Note!*
To use the [CardanoScan API](https://docs.cardanoscan.io/), you must have a valid API key. The `getApiKey` function retrieves this key from your environment variables, so ensure your key is named `CARDANO_API_KEY` or change the code below to ensure your key will be recognized. 

1. **`getApiKey`**:
   - Retrieves your API key from the environment.
   - Ensures the key is securely accessed and avoids hardcoding sensitive information.

2. **`saveJsonToFile`**:
   - Formats the JSON response into a more readable format and saves it to a file.
   - By default, it saves the output to [`collectionassets.json`](./data/collectionassets.json).

3. **`getAssetsByAddress`**:
   - Configures the API request by setting the required headers (including the API key) and query parameters.
   - Sends the request to the API endpoint and handles the response.


In [15]:
type Token = {
    policyId: string
    assetName: string
    fingerprint: string
    assetId: string
    balance: string
}

type ApiResponse = {
    tokens: Token list
    count: int
    pageNo: int
    limit: int
}

In [3]:
let getApiKey () =
    match Environment.GetEnvironmentVariable("CARDANO_API_KEY") with
    | null -> failwith "API key not found. Set it in your environment variables."
    | key -> key

In [19]:
let saveJsonToFile (filePath: string) (jsonResponse: string) =
    try
        
        let data = JsonSerializer.Deserialize<obj>(jsonResponse)
        let options = JsonSerializerOptions(WriteIndented = true, Encoder = System.Text.Encodings.Web.JavaScriptEncoder.UnsafeRelaxedJsonEscaping)
        let jsonOutput = JsonSerializer.Serialize(data, options)

        File.WriteAllText(filePath, jsonOutput)
    with ex ->
        printfn "Failed to save JSON. Error: %s" ex.Message

In [ ]:
let getAssetsByAddress (address: string) (filePath: string) =
    async {
        let apiKey = getApiKey()
        let baseUrl = "https://api.cardanoscan.io/api/v1/asset/list/byAddress"
        let mutable pageNo = 1
        let mutable allTokens = []

        use client = new HttpClient()
        client.DefaultRequestHeaders.Add("apiKey", apiKey)

        try
            let mutable hasMoreData = true

            while hasMoreData do
                let url = $"{baseUrl}?address={address}&pageNo={pageNo}"
                let! response = client.GetAsync(url) |> Async.AwaitTask

                if response.IsSuccessStatusCode then
                    let! content = response.Content.ReadAsStringAsync() |> Async.AwaitTask
                    try
                        let data = JsonSerializer.Deserialize<ApiResponse>(content)

                        if data.tokens.IsEmpty then
                            hasMoreData <- false
                        else
                            allTokens <- allTokens @ [data]
                            pageNo <- pageNo + 1
                    with ex ->
                        printfn "JSON Deserialization Error: %s" ex.Message
                        printfn "Raw Content: %s" content
                        return Error "Failed to deserialize API response." |> ignore
                else
                    let! errorContent = response.Content.ReadAsStringAsync() |> Async.AwaitTask
                    printfn "API Error: %s" errorContent
                    return Error $"Error: {response.StatusCode} - {response.ReasonPhrase}" |> ignore

            let allTokensJson = JsonSerializer.Serialize(allTokens, JsonSerializerOptions(WriteIndented = true))
            saveJsonToFile filePath allTokensJson

            printfn "Successfully saved all tokens to %s" filePath
            return Ok ()
        with ex ->
            printfn "Exception occurred: %s" ex.Message
            return Error $"Exception: {ex.Message}"
    }


## Enter the Address and Call the Functions

Replace the placeholder address below to retrieve and save the assets associated with a Cardano address. Please note that the endpoint requires the **hex representation of the address**.


In [17]:
let address = "PlaceAddressHere" 

In [ ]:
let filePath = "./data/collectionassets.json"

async {
    let! result = getAssetsByAddress address filePath
    match result with
    | Ok () -> ()
    | Error errMsg -> printfn "Error: %s" errMsg
}
|> Async.RunSynchronously

Successfully saved all tokens to ./data/collectionassets.json
API response saved successfully.
